In [25]:
### string.fromat

import yaml
import import_ipynb
import Read_pdf as rp
from tkinter import simpledialog
import tkinter as tk

def display_template(data):
    topic = data.keys()
    print('--------------模板列表-----------------')
    for item in topic:
        print(item + ':')
        for sub_item in data[item]:
            print(' -' + sub_item)

def get_variable(text):#抓取{ 及 }的位置並取出裏頭的字串變成dic的key
    my_pos = []
    my_pos2 = []
    pos = 0
    pos2 = 0
    while True:
        pos = text.find('{', pos)
        pos2 = text.find('}', pos2)
        if pos2 == -1:
            break
        my_pos.append(pos)
        my_pos2.append(pos2)
        pos += 1
        pos2 +=1

    keys = []
    for start,end in zip(my_pos, my_pos2):
        keys.append(text[start+1:end])
    d = dict.fromkeys(keys)
    del my_pos
    del my_pos2
    del keys
    return d


def bid_value(bid, dic):
    if bid.currency == '人民幣':
        prefixCur = 'RMB'
        suffixCur = ''
    else:
        prefixCur =''
        suffixCur = '元'

    def name_format(var):
        return '「' + str(var) +'」'

    def price_format(var):
        return prefixCur + str(var) + suffixCur
    
    select_unNone = lambda final, quo : final if final != None else quo
    
    for key in dic:
        if key == '次別':
            dic[key] = bid.sn
        elif key == '決包廠商':
            dic[key] = name_format(bid.award_vnd)
        elif key == '追加減':
            dic[key] = bid.change
        elif key == '公司別':
            dic[key] = bid.comp
        elif key == '廠區':
            dic[key] = bid.id[1:3]
        elif key == '預算金額':
            dic[key] = price_format(bid.budget)
        elif key == '報價金額':
            dic[key] = price_format(bid.vnd[bid.cheapest_sn]['quo'])
        elif key == '最後金額':
            dic[key] = price_format(select_unNone(bid.vnd[bid.cheapest_sn]['quo'], \
                                                  bid.vnd[bid.cheapest_sn]['finalPrice']))
        elif key == '議價金額':
            if '由/維持原報價' in dic and dic['由/維持原報價'] == '維持原報價':
                dic[key] = ''
            else:
                dic[key] = price_format(bid.vnd[bid.cheapest_sn]['finalPrice'])
            
        elif key == '差異':
            dic[key] = bid.dif_text(bid.cheapest_sn)
        elif key == '工程名稱':
            dic[key] = bid.name
        elif key == '最低廠商':
            dic[key] = name_format(bid.vnd[bid.cheapest_sn]['name'])
        elif key == '報價家數':
                dic[key] = len(bid.vnd)
        elif key == '及催報':
            dic[key] = ''
            for vnd in bid.vnd:
                if vnd['type'] == '後報' or vnd['type'] == '後加':
                    dic[key] = '及催報'
                    break
        
        elif key == '議價狀況':
            sum_bargain_times = 0 #議價總次數
            for vnd in bid.vnd:
                sum_bargain_times += int(vnd['bargain_times'])
                
            if bid.cheapest_sn == 0 and int(bid.vnd[bid.cheapest_sn]['bargain_times']) == sum_bargain_times:
                dic[key] = '經洽議報價最低廠商'
            else:
                dic[key] = '議後以廠商'
        elif key == '議後':
            if bid.vnd[bid.cheapest_sn]['bargain_times'] != '0':
                dic[key] = '議後'
            else:
                dic[key] = ''
        elif key == '由/維持原報價':
            if bid.vnd[bid.cheapest_sn]['quo'] == bid.vnd[bid.cheapest_sn]['finalPrice']:
                dic[key] = '維持原報價'
            else:
                dic[key] = '由'
        elif key == '降至':
            if dic['由/維持原報價'] == '維持原報價':
                dic[key] = ''
            else:
                dic[key] = '降至'
        
        elif key == '最低':
            if dic['議價狀況'] == '議後以廠商':
                dic[key] = '最低'
            else:
                dic[key] = ''
        
        elif key == '小規模納稅人':
            rate = select_unNone(bid.vnd[bid.cheapest_sn]['finalRate'], bid.vnd[bid.cheapest_sn]['quoRate'])
            if int(rate) == 3 :
                dic[key] = '小規模納稅人'
            else:
                dic[key] = ''
        
        elif key == '廠商稅率':
            rate = select_unNone(bid.vnd[bid.cheapest_sn]['finalRate'], bid.vnd[bid.cheapest_sn]['quoRate'])
            dic[key] = rate
            
        elif key =='發票符合/洽會同意':
            setting_rate = dic['預算稅率']
            rate = select_unNone(bid.vnd[bid.cheapest_sn]['finalRate'], bid.vnd[bid.cheapest_sn]['quoRate'])
            
            if setting_rate == bid.vnd[bid.cheapest_sn]['finalRate']:
                dic[key] = '符合'
            elif rate == '3' :
                dic[key] = ''
            else:
                dic[key] = '(洽會工程部門確認同意)'
        
        elif key == '回議':
            if bid.cheapest_sn == 0:
                dic[key] = ''
            else:
                if bid.vnd[0]['finalPrice'] == None:
                    dic[key] = '，回議廠商{name}未回覆，不再回議'.format(name = name_format(bid.vnd[0]['name']))
                else:
                    dic[key] = '，回議廠商{name}回覆{price}價高'.format(name = name_format(bid.vnd[0]['name']),\
                                                              price = price_format(bid.vnd[0]['finalPrice']))
        elif key == '差異原因':
            dic[key] = '{差異原因}'
        else:
            root = tk.Tk()
            root.withdraw()
            dic[key] = simpledialog.askstring('套版工具', '請輸入「{}」的值:'.format(key))
            root.destroy()

    return dic

def use_template(bidcase , easymode = None):
    if easymode == None:
        easymode = False    

    is_bargain = lambda n :'有議價' if n else '無議價'
    is_overbudget = lambda n :'(超預算)' if n else '(低預算)'


    a = ''
    my_mode = is_bargain(bidcase.is_bargain)

    if bidcase.type == '一般案' and bidcase.is_bargain:
        a = is_overbudget(bidcase.is_overBudget(bidcase.cheapest_sn))
    elif easymode and bidcase.type != '一般案':
        my_mode = '簡易版'

    print('--------------模板樣式[{type}][{mode}]-------------'\
        .format(type = bidcase.type, mode = my_mode + a))

    text = data[bidcase.type][my_mode + a]
    print(text)

    dict_tags = get_variable(text)
    dict_tags = bid_value(bidcase, dict_tags)

    text = text.format(**dict_tags) 
    return text

if __name__ == '__main__':
        
    file_yaml = 'template.yaml'
    with open(file_yaml, 'r', encoding = 'utf8') as stream:
        data = yaml.load(stream, Loader = yaml.FullLoader)
    s
#     display_template(data)        

    pdf_file = 'd:/N2.pdf'
    bidcase = rp.open(pdf_file)
    
    text = use_template(bidcase)
    print('----------------套版後-------------------')
    print(text)
    

--------------模板樣式[一般案][有議價(超預算)]-------------
1.本案為{公司別}{廠區}「{工程名稱}」，預算編列{預算金額}，增值稅率需求為{預算稅率}%。
2.開標{及催報}後共計{報價家數}家廠商報價，{議價狀況}{最低廠商}{由/維持原報價}{報價金額}{降至}{議價金額}{最低}，{差異}{回議}。廠商反應本案{差異原因}。
3.擬擇廉以{議價金額}交{最低廠商}承攬，{小規模納稅人}開立{廠商稅率}%發票{發票符合/洽會同意}。

----------------套版後-------------------
1.本案為2NJJ3「昆山CCL三廠配料室T-316至含浸配管」，預算編列RMB 89,262，增值稅率需求為9%。
2.開標及催報後共計4家廠商報價，議後以廠商「江苏江安集团」由RMB 131,462降至RMB 116,000最低，較預算+29.95%。廠商反應本案{差異原因}。
3.擬擇廉以RMB 116,000交「江苏江安集团」承攬，開立9%發票符合。

